This kernel borrows the data preprocessing part from this very thorough and useful kernel by Lathwal: https://www.kaggle.com/codename007/forest-cover-type-eda-baseline-model

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from collections import Counter
import matplotlib.pyplot as plt
#import seaborn as sns
from mlxtend.classifier import StackingCVClassifier
from sklearn.linear_model import LogisticRegression
%matplotlib inline
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
from pandas.tools.plotting import parallel_coordinates
from sklearn.decomposition import PCA
from sklearn import ensemble,model_selection,svm
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
print('size of train data',train.shape)
print('size of test data',test.shape)

In [ ]:
train.head()

In [ ]:
train.info()

# Feature Engineering

* **Features engineering** :
    * **Linear combination** :
           * Elevation, Vertical dist. to Hydrology
           * Horizontal dist. to Hydrology, Horizontal dist. to Fire Points 
           * Horizontal dist. to Hydrology, Horizontal dist. to Roadways 
           * Horizontal dist. to Fire Points, Horizontal dist. to Roadways 
    * **Euclidean distance** : 
            * Horizontal dist. to Hydrology, Vertical dist. to Hydrology
            * euclidean distance = \sqrt{(verticaldistance)^2 + (horizontaldistance)^2}\] 
    * **Distance to Amenities** :
         * As we know distances to amenities like Water, Fire and Roadways played a key role in determining cover type 
           * Mean distance to Amenities 
           * Mean Distance to Fire and Water 

In [ ]:
####################### Train data #############################################
train['HF1'] = train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Fire_Points']
train['HF2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Fire_Points'])
train['HR1'] = abs(train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Roadways'])
train['HR2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Roadways'])
train['FR1'] = abs(train['Horizontal_Distance_To_Fire_Points']+train['Horizontal_Distance_To_Roadways'])
train['FR2'] = abs(train['Horizontal_Distance_To_Fire_Points']-train['Horizontal_Distance_To_Roadways'])
train['ele_vert'] = train.Elevation-train.Vertical_Distance_To_Hydrology

train['slope_hyd'] = (train['Horizontal_Distance_To_Hydrology']**2+train['Vertical_Distance_To_Hydrology']**2)**0.5
train.slope_hyd=train.slope_hyd.map(lambda x: 0 if np.isinf(x) else x) # remove infinite value if any

#Mean distance to Amenities 
train['Mean_Amenities']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology + train.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
train['Mean_Fire_Hyd']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology) / 2 

####################### Test data #############################################
test['HF1'] = test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Fire_Points']
test['HF2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Fire_Points'])
test['HR1'] = abs(test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Roadways'])
test['HR2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Roadways'])
test['FR1'] = abs(test['Horizontal_Distance_To_Fire_Points']+test['Horizontal_Distance_To_Roadways'])
test['FR2'] = abs(test['Horizontal_Distance_To_Fire_Points']-test['Horizontal_Distance_To_Roadways'])
test['ele_vert'] = test.Elevation-test.Vertical_Distance_To_Hydrology

test['slope_hyd'] = (test['Horizontal_Distance_To_Hydrology']**2+test['Vertical_Distance_To_Hydrology']**2)**0.5
test.slope_hyd=test.slope_hyd.map(lambda x: 0 if np.isinf(x) else x) # remove infinite value if any

#Mean distance to Amenities 
test['Mean_Amenities']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology + test.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
test['Mean_Fire_Hyd']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology) / 2

# Model Building

In [ ]:
feature = [col for col in train.columns if col not in ['Cover_Type','Id']]
X_train = train[feature]
X_test = test[feature]
c1 = ensemble.ExtraTreesClassifier(n_estimators=150,bootstrap=True) 
c2= ensemble.RandomForestClassifier(n_estimators=150,bootstrap=True)
c3=XGBClassifier();
meta = svm.LinearSVC()
etc = StackingCVClassifier(classifiers=[c1,c2,c3],use_probas=True,meta_classifier=meta)

#parameters = {'n_estimators':np.range(100,500)}
#gc=model_selection.GridSearchCV(parameters,etc)
#etc=XGBClassifier();stackcv
etc.fit(X_train.values, train['Cover_Type'].values)
sub = pd.DataFrame({"Id": test['Id'],"Cover_Type": etc.predict(X_test.values)})
sub.to_csv("stackcv_linearsvc.csv", index=False) 